In [1]:
import altair as alt
import glob
import pandas as pd
from tqdm import tqdm
from parse_science_log import parse

In [2]:
log = parse('PROJ17100/RUN0/CLONE0/results0/science.log')

In [3]:
log.get_active_device()

Device(name='GeForce GTX 1080 Ti', vendor='NVIDIA Corporation', version='OpenCL 1.2 CUDA')

In [4]:
log.fah_core_log.average_perf_ns_day

638.783

In [5]:
files = glob.glob('PROJ17100/RUN0/*/results0/science.log')
len(files)

416

In [6]:
def get_gpu_perf(log_file):
    try:
        p = parse(log_file)
        return p.get_active_device().name, p.fah_core_log.average_perf_ns_day
    except:
        return None

gpu_perf = pd.DataFrame.from_records([get_gpu_perf(f) for f in tqdm(files)],  columns=["gpu_name", "ns_per_day"])

100%|██████████| 416/416 [00:56<00:00,  7.40it/s]


In [7]:
gpu_perf.head()

,gpu_name,ns_per_day
0,Intel(R) UHD Graphics 630,41.1553
1,gfx906,439.8940
2,GeForce GTX 1050 Ti,192.7630
3,Intel(R) UHD Graphics 630,43.9251
4,GeForce GTX 1050 Ti,157.8070


In [8]:
alt.Chart(gpu_perf).mark_bar().encode(
    x="mean(ns_per_day)",
    y=alt.Y("gpu_name:N", sort='-x')
)

alt.Chart(...)